In [1]:
# Imports
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from tqdm import tqdm

import os
import pickle

In [2]:
# Load OpenAI's API key
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
# Load documents' texts
with open("../dataset/documents.pkl", "rb") as f:
    docs = pickle.load(f)

In [4]:
# Load queries' text
with open("../dataset/queries.pkl", "rb") as f:
    queries = pickle.load(f)

In [6]:
# Load query and its most relevant documents
with open("../ir_techniques/exact_search/index.pkl", "rb") as f:
    predictions = pickle.load(f)

In [7]:
# Load ground truth
with open("../dataset/rel_set.pkl", "rb") as f:
    rel_set = pickle.load(f)

In [8]:
k = 10  # Choose the top-k most relevant documents

### gpt-3.5-turbo-instruct

In [9]:
# Define the LLM
gpt_llm = OpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo-instruct")

In [10]:
# Define the prompt for the LLM
template = """Answer the question or Explain the topic given this additional context: {context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [11]:
# Define the RAG pipeline
gpt_chain = ({"context": RunnablePassthrough(), "question": RunnablePassthrough()} 
             | prompt 
             | gpt_llm 
             | StrOutputParser())

In [12]:
# Test a query
# mquery_id = 1
# context_str = "\n\n\n".join([docs[doc_id]["text"] for doc_id in predictions[mquery_id][:k]])[:4096]
# input_data = {"context": context_str, "question": queries[mquery_id]["text"]}
# gpt_chain.invoke(input_data)

In [13]:
#### API CALL WARNING #####

# Run RAG pipeline for every query
answers_gpt = {}
for query_id in tqdm(rel_set.keys(), desc="Asking Queries to ChatGPT with RAG"):
    context_str = "\n\n\n".join([docs[doc_id]["text"] for doc_id in predictions[query_id][:k]])[:4096]
    input_data = {"context": context_str, "question": queries[query_id]["text"]}
    response = gpt_chain.invoke(input_data)
    answers_gpt[query_id] = {"response": response}

Asking Queries to ChatGPT with RAG:   0%|                             | 0/76 [00:02<?, ?it/s]


In [14]:
# Sanity check
queries[1]["text"], answers_gpt[1]["response"]

(' What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?',
 "Answer: The main concern with creating descriptive titles is that they may not accurately represent the content of the article. This can make it difficult for users to find the information they are looking for when using automated retrieval methods. The usual relevance of the content of articles to their titles is an important factor in determining the effectiveness of a citation index. This is because it allows users to quickly identify relevant articles based on the title alone, without having to read the entire article. However, this relevance can be subjective and may vary depending on the individual's search requirements and judgment.")

In [15]:
# Save all responses
with open("../responses/gpt-3.5-turbo-instruct/llm_w_rag_exact_search.pkl", "wb") as f:
    pickle.dump(answers_gpt, f)